# 04 — Model Evaluation & Error Analysis

**Project:** Early ICU Mortality Prediction Using Structured EHR Data  
**Dataset:** MIMIC-IV Clinical Database Demo (v2.2)

## Goal of this notebook
Evaluate the baseline mortality model from `03_train_baseline_model.ipynb` and understand its behavior.

We will:
1. Load the saved model and metrics
2. Recompute predictions on a **patient-level** held-out split (to reduce leakage)
3. Plot:
   - ROC curve
   - Precision–Recall curve
   - Calibration curve
   - Predicted probability distributions by class
4. Do threshold analysis
5. Perform simple error analysis (top false positives / false negatives + feature contribution breakdown)
6. (Optional) Subgroup checks if `patients.csv` is available (e.g., by sex, age bins)

## Inputs
- `baseline_logreg.joblib`
- `baseline_metrics.json` (optional, for comparison)
- `dataset_model_ready.csv`

## Outputs
- `eval_predictions.csv` (test split predictions)
- `threshold_report.csv`


In [ ]:
# Setup
import pandas as pd
import numpy as np
from pathlib import Path

pd.set_option("display.max_columns", 200)
pd.set_option("display.width", 120)

import sys, platform
print("Python:", sys.version.split()[0])
print("Platform:", platform.platform())
print("Pandas:", pd.__version__)


## 1) Load artifacts (dataset, model, optional metrics)

We use a small helper to resolve paths whether you run this notebook:
- in the same folder as the CSVs, or
- in an environment where files are in `/mnt/data`.


In [ ]:
def resolve_dir():
    d = Path(".")
    if (d / "dataset_model_ready.csv").exists():
        return d
    alt = Path("/mnt/data")
    if (alt / "dataset_model_ready.csv").exists():
        return alt
    return d

DATA_DIR = resolve_dir()
print("Using DATA_DIR:", DATA_DIR.resolve())

DATASET_PATH = DATA_DIR / "dataset_model_ready.csv"
MODEL_PATH = DATA_DIR / "baseline_logreg.joblib"
METRICS_PATH = DATA_DIR / "baseline_metrics.json"

assert DATASET_PATH.exists(), f"Missing {DATASET_PATH}"
assert MODEL_PATH.exists(), f"Missing {MODEL_PATH}"

df = pd.read_csv(DATASET_PATH)

import joblib, json
model = joblib.load(MODEL_PATH)

saved_metrics = None
if METRICS_PATH.exists():
    with open(METRICS_PATH, "r") as f:
        saved_metrics = json.load(f)

print("Loaded dataset:", df.shape)
print("Loaded model:", type(model))
if saved_metrics:
    print("Found baseline_metrics.json ✅")
    display(pd.DataFrame([saved_metrics]))
else:
    print("baseline_metrics.json not found (that's okay).")

display(df.head(5))


## 2) Define label and feature columns

- **Label:** `label_mortality`
- **Features:** columns starting with `lab_`

We exclude identifiers and timestamps from modeling features.


In [ ]:
LABEL_COL = "label_mortality"
feature_cols = [c for c in df.columns if c.startswith("lab_")]

assert LABEL_COL in df.columns, f"Missing label column: {LABEL_COL}"
assert len(feature_cols) > 0, "No feature columns found (expected columns starting with 'lab_')"

# Clean label
df = df[df[LABEL_COL].notna()].copy()
df[LABEL_COL] = df[LABEL_COL].astype(int)

# For determinism across runs
sort_cols = [c for c in ["subject_id", "hadm_id", "stay_id"] if c in df.columns]
df = df.sort_values(sort_cols).reset_index(drop=True)

X = df[feature_cols]
y = df[LABEL_COL]
groups = df["subject_id"]

print("Rows:", len(df))
print("Features:", len(feature_cols))
print("Label distribution:")
display(y.value_counts())


## 3) Recreate a patient-level train/test split

We use the same strategy as the baseline notebook:
- **GroupShuffleSplit** by `subject_id` (patient)

This avoids putting multiple ICU stays from the same patient into both train and test.


In [ ]:
from sklearn.model_selection import GroupShuffleSplit

gss = GroupShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
train_idx, test_idx = next(gss.split(X, y, groups=groups))

X_test = X.iloc[test_idx]
y_test = y.iloc[test_idx]
df_test = df.iloc[test_idx].copy()

print("Test rows:", len(df_test))
print("Test label distribution:", y_test.value_counts().to_dict())


## 4) Predict and compute headline metrics

In [ ]:
from sklearn.metrics import roc_auc_score, average_precision_score

proba_test = model.predict_proba(X_test)[:, 1]

roc = roc_auc_score(y_test, proba_test) if y_test.nunique() > 1 else np.nan
pr  = average_precision_score(y_test, proba_test) if y_test.nunique() > 1 else np.nan

print(f"ROC-AUC: {roc:.3f}")
print(f"PR-AUC (Average Precision): {pr:.3f}")

df_test["pred_proba"] = proba_test
display(df_test[["subject_id", "hadm_id", "stay_id", LABEL_COL, "pred_proba"]].head(10))


## 5) ROC curve

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve

fpr, tpr, _ = roc_curve(y_test, proba_test)

plt.figure()
plt.plot(fpr, tpr)
plt.plot([0, 1], [0, 1], linestyle="--")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("ROC Curve (Test Split)")
plt.show()


## 6) Precision–Recall curve

In [ ]:
from sklearn.metrics import precision_recall_curve

precision, recall, _ = precision_recall_curve(y_test, proba_test)

plt.figure()
plt.plot(recall, precision)
plt.xlabel("Recall")
plt.ylabel("Precision")
plt.title("Precision–Recall Curve (Test Split)")
plt.show()


## 7) Calibration curve (reliability)

Calibration answers: *when the model says 0.7 risk, does ~70% actually happen?*

With small demo data, calibration will be noisy — treat this as a diagnostic, not a final statement.


In [ ]:
from sklearn.calibration import calibration_curve

prob_true, prob_pred = calibration_curve(y_test, proba_test, n_bins=5, strategy="quantile")

plt.figure()
plt.plot(prob_pred, prob_true, marker="o")
plt.plot([0, 1], [0, 1], linestyle="--")
plt.xlabel("Mean Predicted Probability")
plt.ylabel("Fraction of Positives")
plt.title("Calibration Curve (Test Split)")
plt.show()

cal_df = pd.DataFrame({"bin_mean_pred": prob_pred, "bin_frac_pos": prob_true})
display(cal_df)


## 8) Predicted probability distributions by class

This helps visualize separation (or overlap) between survivors and non-survivors.


In [ ]:
plt.figure()
plt.hist(df_test.loc[df_test[LABEL_COL] == 0, "pred_proba"].dropna(), bins=10, alpha=0.8, label="label=0")
plt.hist(df_test.loc[df_test[LABEL_COL] == 1, "pred_proba"].dropna(), bins=10, alpha=0.8, label="label=1")
plt.xlabel("Predicted probability")
plt.ylabel("Count")
plt.title("Predicted Probabilities by Class (Test Split)")
plt.legend()
plt.show()


## 9) Threshold analysis

We compute precision/recall/F1 across thresholds, and highlight:
- `threshold = 0.5` (default)
- best-F1 threshold (on the test split, for diagnostics)

> In real settings, threshold choice should reflect the cost of false positives vs false negatives.


In [ ]:
from sklearn.metrics import confusion_matrix

def metrics_at_threshold(y_true, proba, thr):
    pred = (proba >= thr).astype(int)
    tn, fp, fn, tp = confusion_matrix(y_true, pred, labels=[0, 1]).ravel()
    precision = tp / (tp + fp + 1e-12)
    recall = tp / (tp + fn + 1e-12)
    f1 = 2 * precision * recall / (precision + recall + 1e-12)
    return {
        "threshold": thr,
        "tp": int(tp), "fp": int(fp), "tn": int(tn), "fn": int(fn),
        "precision": float(precision),
        "recall": float(recall),
        "f1": float(f1),
    }

thresholds = np.linspace(0.05, 0.95, 19)
report = pd.DataFrame([metrics_at_threshold(y_test.values, proba_test, float(t)) for t in thresholds])

best_row = report.loc[report["f1"].idxmax()]
print("Best-F1 threshold on test split:", float(best_row["threshold"]))

display(report.sort_values("f1", ascending=False).head(10))


## 10) Error analysis: top false positives / false negatives

We list the most confident mistakes on the test split:
- False positives: high predicted risk but survived
- False negatives: low predicted risk but died

Then we provide a small helper to explain *which features* contribute most to the prediction for a specific stay.


In [ ]:
# Label predictions at a chosen threshold (use 0.5 for consistency)
THRESHOLD = 0.5
df_test["pred_label"] = (df_test["pred_proba"] >= THRESHOLD).astype(int)

fp = df_test[(df_test[LABEL_COL] == 0) & (df_test["pred_label"] == 1)].sort_values("pred_proba", ascending=False)
fn = df_test[(df_test[LABEL_COL] == 1) & (df_test["pred_label"] == 0)].sort_values("pred_proba", ascending=True)

print("False positives:", len(fp))
print("False negatives:", len(fn))

display(fp[["subject_id", "hadm_id", "stay_id", LABEL_COL, "pred_proba"]].head(10))
display(fn[["subject_id", "hadm_id", "stay_id", LABEL_COL, "pred_proba"]].head(10))


### Explain a prediction (feature contributions)

For logistic regression, we can decompose the log-odds into per-feature contributions:
- Transform features through the model’s preprocessing (imputer + scaler)
- Multiply by the classifier coefficients

This is a **debugging aid**, not a clinical explanation.


In [ ]:
def explain_stay(stay_id, top_k=10):
    # Extract single row
    row = df_test[df_test["stay_id"] == stay_id]
    if row.empty:
        raise ValueError(f"stay_id {stay_id} not found in test split")
    x = row[feature_cols]

    # Transform through preprocessing
    x_imp = model.named_steps["imputer"].transform(x)
    x_scaled = model.named_steps["scaler"].transform(x_imp)

    coef = model.named_steps["clf"].coef_.ravel()
    intercept = float(model.named_steps["clf"].intercept_.ravel()[0])

    contrib = x_scaled.ravel() * coef
    contrib_df = pd.DataFrame({
        "feature": feature_cols,
        "value_raw": x.iloc[0].values,
        "contribution_to_logodds": contrib,
        "abs_contribution": np.abs(contrib)
    }).sort_values("abs_contribution", ascending=False)

    # Reconstruct logit/prob (sanity check)
    logit = intercept + contrib.sum()
    prob = 1 / (1 + np.exp(-logit))

    meta = row[["subject_id", "hadm_id", "stay_id", LABEL_COL, "pred_proba"]].copy()
    meta["reconstructed_prob"] = prob

    return meta, contrib_df.head(top_k)

# Example: explain the most confident false positive (if any)
if len(fp) > 0:
    example_stay = int(fp.iloc[0]["stay_id"])
    meta, top_contrib = explain_stay(example_stay, top_k=12)
    display(meta)
    display(top_contrib)
else:
    print("No false positives at threshold=0.5 to explain.")


## 11) Optional subgroup checks (if `patients.csv` is available)

If you have `patients.csv` in the same folder, we can join it to get:
- `gender`
- `anchor_age` (de-identified age)

Then compute metrics by subgroup.

This section is optional and will skip automatically if the file is not present.


In [ ]:
PATIENTS_PATH = DATA_DIR / "patients.csv"
if PATIENTS_PATH.exists():
    patients = pd.read_csv(PATIENTS_PATH)[["subject_id", "gender", "anchor_age"]]
    df_test_sub = df_test.merge(patients, on="subject_id", how="left")

    # Age bins (rough; demo is small)
    df_test_sub["age_bin"] = pd.cut(df_test_sub["anchor_age"], bins=[0, 40, 60, 80, 120], right=False)

    def group_metrics(frame):
        yt = frame[LABEL_COL].values
        pt = frame["pred_proba"].values
        out = {
            "n": len(frame),
            "positives": int(frame[LABEL_COL].sum()),
            "roc_auc": float(roc_auc_score(yt, pt)) if len(np.unique(yt)) > 1 else np.nan,
            "pr_auc": float(average_precision_score(yt, pt)) if len(np.unique(yt)) > 1 else np.nan,
        }
        return pd.Series(out)

    by_gender = df_test_sub.groupby("gender", dropna=False).apply(group_metrics).reset_index()
    by_age = df_test_sub.groupby("age_bin", dropna=False).apply(group_metrics).reset_index()

    print("Metrics by gender:")
    display(by_gender)

    print("Metrics by age_bin:")
    display(by_age)
else:
    print("patients.csv not found; skipping subgroup checks.")


## 12) Save evaluation artifacts

We save:
- `eval_predictions.csv`: identifiers + true label + predicted probability on the test split
- `threshold_report.csv`: precision/recall/F1 across thresholds


In [ ]:
PRED_OUT = Path("eval_predictions.csv")
THR_OUT = Path("threshold_report.csv")

df_test_out = df_test[["subject_id", "hadm_id", "stay_id", LABEL_COL, "pred_proba"]].copy()
df_test_out.to_csv(PRED_OUT, index=False)
report.to_csv(THR_OUT, index=False)

print("Saved:")
print(" ", PRED_OUT.resolve())
print(" ", THR_OUT.resolve())
